In [3]:
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import lit
import os
file_list = [f'data/{f}' for f in os.listdir('data')]
print(file_list)
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("GenericAppName") \
    .getOrCreate()
df = spark.read.csv(file_list[0], header=True, inferSchema=True)
year = int('20' + file_list[0].split('_')[-1].split('.')[0])
df = df.withColumn('year', lit(year))
for file_path in file_list[1:]:
    cur_df = spark.read.csv(file_path, header=True, inferSchema=True)
    year = int('20' + file_path.split('_')[-1].split('.')[0])
    cur_df = cur_df.withColumn('year', lit(year))
    df = df.unionByName(cur_df)
df.show()



['data/players_22.csv', 'data/players_21.csv', 'data/players_20.csv', 'data/players_18.csv', 'data/players_19.csv', 'data/players_17.csv', 'data/players_16.csv', 'data/players_15.csv']


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/19 23:50:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/19 23:50:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


22/10/19 23:50:30 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+---------+--------------------+-----------------+--------------------+----------------+-------+---------+---------+--------+---+-------------------+---------+---------+------------+-------------------+--------------------+------------+-------------+------------------+----------------+-------------------+-------------------------+--------------+----------------+--------------+---------------+--------------------+--------------+---------+-----------+------------------------+-------------+----------------+---------+------------------+--------------------+--------------------+----+--------+-------+---------+---------+------+------------------+-------------------+--------------------------+-----------------------+-----------------+---------------+-----------+-----------------+------------------+------------------

In [4]:
db_properties = {}
# update your db username
db_properties['username'] = "postgres"
# update your db password
db_properties['password'] = "root"
# make sure you got the right port number here
db_properties['url'] = "jdbc:postgresql://localhost:5432/postgres"
# make sure you had the Postgres JAR file in the right location
db_properties['driver'] = "org.postgresql.Driver"
db_properties['table'] = "fifa"

df.write.format("jdbc")\
    .mode("overwrite")\
    .option("url", db_properties['url'])\
    .option("dbtable", db_properties['table'])\
    .option("user", db_properties['username'])\
    .option("password", db_properties['password'])\
    .option("Driver", db_properties['driver'])\
    .save()


In [5]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

df_read = sqlContext.read.format("jdbc")\
    .option("url", db_properties['url'])\
    .option("dbtable", db_properties['table'])\
    .option("user", db_properties['username'])\
    .option("password", db_properties['password'])\
    .option("Driver", db_properties['driver'])\
    .load()

df_read.show(1, vertical=True)


/Users/congtan/Desktop/2022Fall/18813/Project1/course-project-option-1-PenguinBoyTC/env/lib/python3.10/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


-RECORD 0-------------------------------------------
 sofifa_id                   | 244554               
 player_url                  | https://sofifa.co... 
 short_name                  | J. Dodd              
 long_name                   | James Dodd           
 player_positions            | CM                   
 overall                     | 53                   
 potential                   | 66                   
 value_eur                   | 100000.0             
 wage_eur                    | 700.0                
 age                         | 18                   
 dob                         | 2000-10-27 00:00:00  
 height_cm                   | 178                  
 weight_kg                   | 66                   
 club_team_id                | 143.0                
 club_name                   | Exeter City          
 league_name                 | English League Two   
 league_level                | 4                    
 club_position               | RES            